In [1]:
# import ray
# ray.init()
# ray.available_resources()
# ray.nodes()

In [2]:
import pandas as pd
import os
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=8)
# os.environ["MODIN_ENGINE"] = "dask"

# import modin.pandas as pd
import numpy as np
import re
import pke

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


import gzip, pickle, pickletools

with open('lda-by-section-text-1-gram-100-topics.pkl', 'rb') as f:
    lda_model = pickle.load(f)
    
with open('tf-wordnet-0.9-2-1-gram.pkl', 'rb') as f:
    tf_model = pickle.load(f)

vocab = tf_model.get_feature_names()

pke_model = (tf_model.get_feature_names(),
                   lda_model.components_,
                   lda_model.exp_dirichlet_component_,
                   lda_model.doc_topic_prior_)

with gzip.open('lda-by-section-text-1-gram-100-topics.gz', 'wb') as f:
    pickle.dump(pke_model, f)

In [2]:
stopwords = ["0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz"]

In [3]:
df = pd.read_parquet('USCS_Codes.parquet', columns=['section_text'])

In [4]:
df.head()

,section_text
0,"In this title, the term “agency” means a depar..."
1,"In this title, the term “barge” means a non-se..."
2,"In this title, the term “Boundary Line” means ..."
3,"In this title, the term “citizen of the United..."
4,"In this title, the term “consular officer” mea..."


In [5]:
df.shape

(58950, 1)

In [6]:
df['section_text'] = df['section_text'].replace('',np.nan)
df = df.dropna(subset=['section_text'])

In [7]:
df['section_text'] = df['section_text'].str.replace(r'\d+\.\d+','')
df['section_text'] = df['section_text'].str.replace(r'\d+-\d+','')
df['section_text'] = df['section_text'].str.replace(r'\d+','')
df['section_text'] = df['section_text'].str.replace('\(?<ref.*?>','')
df['section_text'] = df['section_text'].str.replace('</ref>\)?','')
df['section_text'] = df['section_text'].str.replace('<table.+?</table>','', flags=re.DOTALL)
df['section_text'] = df['section_text'].str.replace('<\w.+?</\w+>','', flags=re.DOTALL)
df['section_text'] = df['section_text'].str.replace('<\w.+?>','', flags=re.DOTALL)
df['section_text'] = df['section_text'].str.replace('</\w.+?>','', flags=re.DOTALL)

/tmp/ipykernel_18859/3702032980.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['section_text'] = df['section_text'].str.replace(r'\d+\.\d+','')
/tmp/ipykernel_18859/3702032980.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['section_text'] = df['section_text'].str.replace(r'\d+-\d+','')
/tmp/ipykernel_18859/3702032980.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['section_text'] = df['section_text'].str.replace(r'\d+','')
/tmp/ipykernel_18859/3702032980.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['section_text'] = df['section_text'].str.replace('\(?<ref.*?>','')
/tmp/ipykernel_18859/3702032980.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['section_text'] = df['section_text'].str.replace('</ref>\)?

In [8]:
df = df.dropna(subset=['section_text'])
df.shape

(40653, 1)

In [9]:
df.sample(20, random_state=42)

,section_text
36387,"Section of this title, and sections (f) and b..."
55492,Assessments collected under section d of this...
51906,"For the purposes of the Federal Food, Drug, an..."
54005,This subchapter may be cited as the “Minority-...
46107,"Each contract in an amount greater than $, tha..."
52095,\n the annual rate of pay of any individual a...
29958,In the event of a default of a loan guaranteed...
39456,\n The head of each national security laborat...
45866,"unmanufactured articles, materials, and suppl..."
14301,is a citizen or resident of the United States...


In [10]:
sentence = df.loc[39456,'section_text']
sentence = sentence.strip()
sentence

'The head of each national security laboratory.  The Commander of the United States Strategic Command.  be comprised of weapons experts from the laboratory that does not have lead responsibility for fielding the warhead being evaluated;  have access to all surveillance and underground test data for all stockpile systems for use in the independent evaluations;  use all relevant available data to conduct independent calculations; and  pursue independent experiments to support the independent evaluations.  review both the matters covered by the assessments under subsection (a) performed by the head of that laboratory and any independent evaluations conducted by a dual validation team under subsection (c);  subject such matters to challenge; and  submit the results of such review and challenge, together with the findings and recommendations of such team with respect to such review and challenge, to the head of that laboratory.  The results of each such assessment.  an identification of the

extractor = pke.unsupervised.TopicalPageRank()

extractor.load_document(input=sentence,
                        language='en',
                        normalization=None)

pos = {'NOUN', 'PROPN', 'ADJ'}
grammar = "NP: {<ADJ>*<NOUN|PROPN>+}"
extractor.candidate_selection(grammar=grammar)

extractor.candidate_weighting(window=10,
                              pos=pos,
                              lda_model='lda-by-section-text-1-gram-100-topics.gz')

keyphrases = extractor.get_n_best(n=30)

keyphrases

In [11]:
def extract_keywords(sentence):
    try:
        extractor = pke.unsupervised.TopicalPageRank()
        extractor.load_document(input=sentence,
                            language='en',
                            normalization=None)
        pos = {'NOUN', 'PROPN', 'ADJ'}
        grammar = "NP: {<ADJ>*<NOUN|PROPN>+}"
        extractor.candidate_selection(grammar=grammar)
        extractor.candidate_weighting(window=10,
                                  pos=pos,
                                  lda_model='lda-by-section-text-1-gram-100-topics.gz')
        keywords = extractor.get_n_best(n=30)
        return np.asarray(keywords).flatten()
    except Exception as e:
        print(e)
        return np.full(60, np.nan)

In [12]:
%%time
extracted_keywords = df.iloc[:1000]['section_text'].parallel_apply(extract_keywords)

/home/jameslittiebrant/anaconda3/envs/keyword/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:394: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn(
/home/jameslittiebrant/anaconda3/envs/keyword/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:394: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn(
/home/jameslittiebrant/anaconda3/envs/keyword/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:394: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn(
/home/jameslittiebrant/anaconda3/envs/keyword/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:394: UserWarning: Your stop_words may be inconsiste

min() arg is an empty sequence
CPU times: user 1.22 s, sys: 262 ms, total: 1.49 s
Wall time: 2min 27s


In [15]:
len(extracted_keywords)

1000

In [16]:
extracted_keywords[0]

array(['united states government', '0.32452084319474295', 'agency',
       '0.23522798223988686', 'department', '0.1440873738094213',
       'instrumentality', '0.12914915361738719', 'term',
       '0.09613003985405615', 'title', '0.0708846072845056'], dtype='<U32')

In [17]:
%%time
keyword_df = pd.DataFrame.from_records(extracted_keywords)

CPU times: user 13.8 ms, sys: 4.29 ms, total: 18 ms
Wall time: 16.5 ms


In [18]:
keyword_df

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,united states government,0.32452084319474295,agency,0.23522798223988686,department,0.1440873738094213,instrumentality,0.12914915361738719,term,0.09613003985405615,...,None,None,None,None,None,None,None,None,None,None
1,barge,0.305840414561605,term,0.2878989545376075,vessel,0.2203936170567775,title,0.18586701384400994,None,None,...,None,None,None,None,None,None,None,None,None,None
2,boundary line,0.2914548781106909,line,0.18639159200566321,b,0.15638689257624067,section,0.11017287215077232,act,0.10752529901138894,...,None,None,None,None,None,None,None,None,None,None
3,united states,0.24480042645505135,nationality act,0.10710640802738752,natural person,0.08369945166249815,immigration,0.06996853544733139,citizen,0.06505293866751755,...,None,None,None,None,None,None,None,None,None,None
4,united states government,0.30240891688251237,consular officer,0.2734307930406615,officer,0.1796498135823231,employee,0.12428749820775362,term,0.09465959504666037,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,allied health professions personnel,0.13007213802122203,allied health,0.0878621331465079,health care,0.07699790328107378,health,0.06058114811384447,private nonprofit institutions,0.05965938071529713,...,training,0.016835583569350346,disease,0.015998708682247224,grants,0.01585407036403614,problems,0.015848132302927392,processes,0.015277769593156795
996,ruth l. kirschstein national research service ...,0.0942565678676813,research training,0.056428405743127956,clinical research,0.054161407225411615,patient care costs,0.0493173769653614,national institute,0.04717874322824121,...,rural populations,0.020488303807831804,cooperative agreements,0.020424173499368054,special needs,0.018653917739202948,public,0.0179187167150347,cooperative agreement,0.017869635056855043
997,claude d. pepper older americans independence ...,0.15021214076290557,such centers,0.07147978653432456,private nonprofit entities,0.06395848779728777,such center,0.06203905305386691,cooperative agreements,0.05976793869220047,...,training,0.016507826688216917,director,0.01634376190326323,diseases,0.015468890668209698,respect,0.014599411953904141,assistance,0.01326770062762865
998,such awards,0.12048854630796349,senior researchers,0.08752428860954818,awards,0.08610553843450111,junior researchers,0.0818925070743766,such areas,0.07319699540051897,...,centers,0.019003763161680454,title,0.01872232544645817,aging,0.015362852789767697,procedures,0.013256247937296179,selection,0.012525226270423236


In [3]:
keyword_df = pd.concat([pd.read_parquet('keywords-0-15000-1gram-30-1000-topics.parquet'),
                        pd.read_parquet('keywords-15000-30000-1gram-30-1000-topics.parquet'),
                        pd.read_parquet('keywords-30000-end-1gram-30-1000-topics.parquet')],axis=0)

In [5]:
keyword_df.columns = [str(x) for x in keyword_df.columns]

In [6]:
terms = ['term_{}'.format(x) for x in range(30)]
term_value = ['term_value_{}'.format(x) for x in range(30)]

In [9]:
columns = np.asarray(list(zip(terms,term_value))).flatten()

In [10]:
columns

array(['term_0', 'term_value_0', 'term_1', 'term_value_1', 'term_2',
       'term_value_2', 'term_3', 'term_value_3', 'term_4', 'term_value_4',
       'term_5', 'term_value_5', 'term_6', 'term_value_6', 'term_7',
       'term_value_7', 'term_8', 'term_value_8', 'term_9', 'term_value_9',
       'term_10', 'term_value_10', 'term_11', 'term_value_11', 'term_12',
       'term_value_12', 'term_13', 'term_value_13', 'term_14',
       'term_value_14', 'term_15', 'term_value_15', 'term_16',
       'term_value_16', 'term_17', 'term_value_17', 'term_18',
       'term_value_18', 'term_19', 'term_value_19', 'term_20',
       'term_value_20', 'term_21', 'term_value_21', 'term_22',
       'term_value_22', 'term_23', 'term_value_23', 'term_24',
       'term_value_24', 'term_25', 'term_value_25', 'term_26',
       'term_value_26', 'term_27', 'term_value_27', 'term_28',
       'term_value_28', 'term_29', 'term_value_29'], dtype='<U13')

In [11]:
keyword_df.columns = columns

In [13]:
df = pd.read_parquet('USCS_Codes.parquet')

In [14]:
df.head()

,chapter_id,chapter_number,chapter_header,section_id,section_number,section_header,section_text
0,/us/usc/t46/stI/ch1,1,DEFINITIONS,/us/usc/t46/s101,101,Agency,"In this title, the term “agency” means a depar..."
1,/us/usc/t46/stI/ch1,1,DEFINITIONS,/us/usc/t46/s102,102,Barge,"In this title, the term “barge” means a non-se..."
2,/us/usc/t46/stI/ch1,1,DEFINITIONS,/us/usc/t46/s103,103,Boundary Line,"In this title, the term “Boundary Line” means ..."
3,/us/usc/t46/stI/ch1,1,DEFINITIONS,/us/usc/t46/s104,104,Citizen of the United States,"In this title, the term “citizen of the United..."
4,/us/usc/t46/stI/ch1,1,DEFINITIONS,/us/usc/t46/s105,105,Consular officer,"In this title, the term “consular officer” mea..."


In [15]:
df['section_text'] = df['section_text'].replace('',np.nan)
df = df.dropna(subset=['section_text'])
df['section_text'] = df['section_text'].str.replace(r'\d+\.\d+','')
df['section_text'] = df['section_text'].str.replace(r'\d+-\d+','')
df['section_text'] = df['section_text'].str.replace(r'\d+','')
df['section_text'] = df['section_text'].str.replace('\(?<ref.*?>','')
df['section_text'] = df['section_text'].str.replace('</ref>\)?','')
df['section_text'] = df['section_text'].str.replace('<table.+?</table>','', flags=re.DOTALL)
df['section_text'] = df['section_text'].str.replace('<\w.+?</\w+>','', flags=re.DOTALL)
df['section_text'] = df['section_text'].str.replace('<\w.+?>','', flags=re.DOTALL)
df['section_text'] = df['section_text'].str.replace('</\w.+?>','', flags=re.DOTALL)

/tmp/ipykernel_36077/432106249.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['section_text'] = df['section_text'].str.replace(r'\d+\.\d+','')
/tmp/ipykernel_36077/432106249.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['section_text'] = df['section_text'].str.replace(r'\d+-\d+','')
/tmp/ipykernel_36077/432106249.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['section_text'] = df['section_text'].str.replace(r'\d+','')
/tmp/ipykernel_36077/432106249.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['section_text'] = df['section_text'].str.replace('\(?<ref.*?>','')
/tmp/ipykernel_36077/432106249.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df['section_text'] = df['section_text'].str.replace('</ref>\)?','')

In [16]:
merged = pd.merge(df, keyword_df, left_index=True, right_index=True)

In [18]:
df.shape

(40653, 7)

In [19]:
merged.to_parquet('USCS Codes Keywords 1000 topics 1gram.parquet')